In [386]:
import fuzzymatcher
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [381]:
def fix_names(name):
    return name.lower().replace('.', '').replace(' ', '_')\
               .replace('_left', '1').replace('_right', '2')

In [29]:
df = pd.read_csv('scrape.csv', parse_dates=['dob'])
df.tail(3)

,D,L,NC,W,dob,fname,height,lname,name,nick,...,sapm,slpm,stance,str. acc,str. def,sub. avg,td acc,td avg,td def,weight
3244,0,2,NaN,11,1978-02-10,Jonathan,75.0,Wiezorek,Jonathan Wiezorek,NaN,...,5.47,1.27,Southpaw,0.58,0.31,0.0,0.20,1.91,0.0,250.0
3245,0,2,NaN,12,1981-07-29,Steve,71.0,Bosse,Steve Bosse,The Boss,...,5.38,5.99,Orthodox,0.52,0.50,0.0,0.33,0.92,1.0,205.0
3246,2,10,NaN,5,1976-07-11,Kenichi,72.0,Yamamoto,Kenichi Yamamoto,NaN,...,1.93,0.30,Orthodox,0.46,0.38,0.3,0.00,0.00,0.1,170.0


In [68]:
df_names = pd.read_csv('sherdog.csv', parse_dates=['birth_date'])\
             .drop(['url', 'fid'], axis=1)\
             .drop_duplicates()\
             .reset_index(drop=True)
df_names.tail(3)

,name,nick,birth_date,height,weight,association,class,locality,country
1526,Ryan Parker,NaN,NaT,75.0,235.0,NaN,Heavyweight,"Moorehead, Minnesota",United States
1527,Marcus Davis,The Irish Hand Grenade,1973-08-24,68.0,170.0,Team Irish,Welterweight,"Bangor, Maine",United States
1528,John Alessio,The Natural,1979-07-05,70.0,155.0,Xtreme Couture,Lightweight,"Vancouver, British Columbia",Canada


In [155]:
# Drop fighters with no stats.
stats = (df['str. acc'] + df['str. def'] + df['sub. avg'] + df['td acc'] 
         + df['td avg'] + df['td def'])
df = df[stats > 0]
df.shape

(2589, 21)

In [328]:
df2 = fuzzymatcher.fuzzy_left_join(df, df_names, ['name'], ['name'])
df2 = df2[df2.best_match_score > .3]

In [397]:
df.rename({'height': 'height_left',
           'name': 'name_left',
           'nick': 'nick_left',
           'weight': 'weight_left'}, axis='columns', inplace=True)
df3 = pd.concat((df2, df), axis=0).drop_duplicates(['fname', 'lname', 'dob'])

In [385]:
df3.drop(['left_id', 'right_id', 'best_match_score', 'name_right'], axis=1,
          inplace=True)
df3.rename(fix_names, axis=1, inplace=True)

In [396]:
df3.reset_index(drop=True, inplace=True)
df3.to_csv('merged.csv', index=False)